In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_blobs
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.optim as optim
import torch.nn.functional import F
import torchvision
from torchvision import transformers
import time
import matplotlib.pyplot as plt

In [ ]:
data = [1.0,2.0,3.0]
tensor = torch.Tensor(data)
print("1D Data: ")
print(tensor)

data = [[1., 2., 3.], [4.,5., 6.]]
tensor = torch.Tensor(data)
print("2D Data: ")
print(tensor)

In [ ]:
rand = torch.randn(2,3,3)
rand

In [ ]:
X = np.arange(0, 10, 0.5)
epsilon = np.random.normal(0, 0.5, len(X))
y = (2 + 3*X)

plt.scatter(X, y, s=20)
plt.title("Randomly Generated Data")
plt.show()

In [ ]:
X, y = X.reshape(-1,1), y.reshape(-1,1)

model = LinearRegression()
model.fit(X, y)
print("W_1:", model.coef_[0][0])
print("W_0:", model.intercept_[0])

In [ ]:
torch.seed()

class SGDRegression(nn.Module):
    def __init__(self):
        self.__init__()
        self.fc1 = nn.Linear(1,1)
    
    def forward(self,x):
        y = self.fc1(x)
        return y

model = SGDRegression()
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.01)
num_epochs = 5000

x_train  = torch.tensor(X,dtype = torch.float32)
y_train = torch.Tensor(y,dtype = torch.float32)



In [ ]:
for name,param in model.named_parameters():
    print(name,param)

In [ ]:
losses = []
for epoch in range(num_epochs):
    y_pred = model(x_train)
    loss = loss_fn(y_pred,y_train)
    losses.append(y_pred,y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
for name,param in model.named_parameters():
    print(name,param)

## Logistic Regression

In [ ]:
centres = [[1,1],[-1,-1]]

x,y = make_blobs(n_samples=750,centers=centres, cluster_std = 0.8, random_state=0)

plt.scatter(x[:,0], x[:,1],c=y)
plt.title("Dummy Classification Data")
plt.plot([-2,2],[2,-2],"r")
plt.show()


y = y.reshape(-1,1)

In [ ]:
torch.seed()

class LogisticRegression(nn.Module):
    def __init__(self):
        super.__init__()
        self.fc1 = nn.Linear(2,1)
    
    def forward(self,x):
        y = torch.sigmoid(self.fc1(x))
        return y

model = LogisticRegression()
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.01)
num_epochs = 5000

x_train = torch.tensor(x,dtype = torch.float32)
y_train = torch.tensor(y,dtype = torch.float32)

In [ ]:
losses = []
for epoch in range(num_epochs):
    y_pred = model(x_train)
    loss = loss_fn(y_pred,y_train)
    losses.append(loss.items())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
accuracy = torch.sum((y_pred>0.5).int()== y_train) / len(y_train)
accuracy